In this notebook we are going to use functionality provided by the SimMLA (pronounced 'Sim-L-A') package to compute the excitation irradiance profile in an epi-illumination fluorescence microscope using a fly's eye condenser. Such a condenser is realized with a pair of microlens arrays (MLA's).

To speed computation, we'll use a 1D array in this example.

In [1]:
%pylab
import SimMLA.fftpack as simfft
import SimMLA.grids   as grids
plt.style.use('dark_background')
plt.rcParams['image.cmap'] = 'plasma'

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


# Setup the MLA geometry
We'll start by defining the geometry of the MLA's.
+ Prior to the objective there are two MLA's with the same parameters.
+ They are spaced by one focal length such that the second MLA is in the focal plane of the first.
+ Each lenslet has the same focal length as all the other lenslets.

In [2]:
numLenslets = 11    # Must be odd; corresponds to the number of lenslets in one dimension
lensletSize = 500   # microns
focalLength = 13700 # microns

wavelength  = 0.642 # microns

Now we'll establish a GridArray to represent the microlenses. Such an array is a coordinate system onto which the individual lenslets are mapped. The GridArray is primarily a convenience class because it manages all unit conversions when performing Fourier transforms.

The most important parameter here is the subgridSize. This is the number of discrete lattice sites into which a lenslet will be divided. The spatial sampling period for each lenslet will therefore be

$$\delta = \frac{p}{N_{sub}}$$

where $p$ is the MLA pitch in microns and $ N_{\text{sub}} $ is the subgrid size, i.e. the number of discrete lattice sites along one dimension of the MLA. For example, a MLA pitch of $p = 500 \, \mu m$ and a subgrid size of $N_{\text{sub}} = 51$ will result in a spatial sampling period of $\frac{500 \, \mu m}{51} \approx 9.8 \mu m$.

A good sampling rate should be smaller than the fastest oscillations in the amplitude and phase of the Fourier transform of the input field to avoid aliasing. Unfortunately, it is impossible to completely avoid aliasing when computing transforms of real apertures, which are not band-limited. We can however minimize their effects by choosing sampling periods as small as possible (or, equivalently, sampling frequencies as large as possible).

We also need to zero pad the input by a large enough amount to minimize rippling effects in the solutions. If we don't zero pad, the numerical Fourier transforms of the input fields are convolved with a sinc whose width is equal to the width of the field. If this occurs, the sinc-based interpolation of the input field's spectrum results in additional artifacts.

In [3]:
subgridSize  = 20001                      # Number of grid (or lattice) sites for a single lenslet
physicalSize = numLenslets * lensletSize # The full extent of the MLA

# dim = 1 makes the grid 1D
grid = grids.GridArray(numLenslets, subgridSize, physicalSize, wavelength, focalLength, dim = 1)

# Define the input field
Next, we'll define an input laser beam that will impinge upon the first MLA.

In [4]:
grid.px.size / numLenslets

20001.0

In [5]:
power   = 100  # mW
beamStd = 1000 # microns
# Gaussian field
# uIn     = lambda x: np.sqrt(power) * (2 * np. pi)**(-0.5) /  beamStd * np.exp(-x**2 / 2 / beamStd**2)

# Plane wave
uIn = lambda x: np.sqrt(power) / physicalSize
uIn = np.vectorize(uIn)

plt.plot(grid.px, uIn(grid.px), linewidth = 2)
plt.xlabel('x-position, microns')
plt.grid(True)
plt.show()

In [6]:
grid.px.size

220011

# Find the field immediately after the second MLA
The field immediately *after* the second MLA is a summation of the Fourier transforms of all the fields sampled by each lenslet in the first array. Each transform is centered around the axis of its corresponding lenslet. Therefore, we have to compute a Fourier transform for each lenslet separately and then shift the origin of its coordinate axes onto the axis for the corresponding lenslet.

The reason that there is no quadratic phase curvature preceding the transforms is because the second MLA acts as a field lens, effectively canceling the quadratic phase terms.

To compute this parallelized Fourier transform, we use SimMLA's fftSubgrid routine. It will return two lists of interpolated fields, one for the magnitude and one for the phase. We will then resample this field onto a new grid representing the coordinate axes immediately after the second MLA.

*The reason we interpolate the resulting fields is because the physical units that the grid is built on change when the Fourier transform is performed. To return results based on the input grid spacing would require downsampling anyway, so I figured it would be better to let the user define the new grid spacing for their needs.*

In [7]:
# Compute the interpolated fields.
%time interpMag, interpPhase = simfft.fftSubgrid(uIn, grid)

CPU times: user 6.62 s, sys: 24 ms, total: 6.65 s
Wall time: 6.65 s


Now that we have the interpolated fields, we'll define a new grid for sampling them. We'll give the grid the same physical extent as before, but we may choose to increase the spatial sampling rate slightly. We will also use the focal length of the objective because it will serve as the final Fourier transforming lens in the system.

*If we increase the sampling rate for the new grid, it won't necessarily fix any subsampling that occurred in the previous step. It will only provide better resolution at the possibly aliased results.*

In [8]:
plt.plot(grid.px, np.abs(interpMag[5](grid.px) * np.exp(1j * interpPhase[5](grid.px))))
plt.show()

In [9]:
%%time
fObj        = 3300 # microns
newGridSize = subgridSize * numLenslets # microns

newGrid = grids.Grid(newGridSize, physicalSize, wavelength, fObj, dim = 1)
field   = np.zeros(newGrid.gridSize)


# For each interpolated magnitude and phase corresponding to a lenslet
# 1) Compute the full complex field
# 2) Sum it with the other complex fields
for currMag, currPhase in zip(interpMag, interpPhase):
    fieldMag   = currMag(newGrid.px)
    fieldPhase = currPhase(newGrid.px)
    
    currField = fieldMag * np.exp(1j * fieldPhase)
    field = field + currField

CPU times: user 528 ms, sys: 0 ns, total: 528 ms
Wall time: 527 ms


In [10]:
fig, (ax0, ax1) = plt.subplots(nrows = 1, ncols = 2, sharey = False)
ax0.plot(newGrid.px, np.abs(field))
ax0.set_xlabel('x-position, microns')

ax1.plot(newGrid.px, np.angle(field))
ax1.set_xlabel('x-position, microns')
plt.show()

In [64]:
np.save('field', field)

In [13]:
plt.plot(newGrid.pX, np.abs(np.fft.fftshift(np.fft.fft(np.fft.fftshift(field))))**2)
plt.show()